<a href="https://colab.research.google.com/github/theindianwriter/CS6910-assignment_2/blob/main/DL_Assign_2_PartB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Libraries

In [1]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
!pip install wandb
import wandb
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

     |████████████████████████████████| 2.1MB 16.2MB/s 
     |████████████████████████████████| 133kB 50.9MB/s 
     |████████████████████████████████| 163kB 57.5MB/s 
     |████████████████████████████████| 102kB 13.1MB/s 
     |████████████████████████████████| 71kB 10.3MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=00dd9b073f286ce204576f076194b5a4be09a6667d9c9d33a34474562feb1634
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=1e81b3bc5f6e157aafa1e3a91f3086e2393e1ca19076beb9813779c9500a21b8
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools
PyTorch Version:  1.8.1+cu101
Torchvision Version:  0.9.1+cu101


#Checking for GPU

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


#Downloading the dataset and removing .DStore 

In [3]:
#we used curlwget to download the data for part B

In [ ]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36 Edg/89.0.774.68" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" "https://storage.googleapis.com/wandb_datasets/nature_12K.zip" -c -O 'nature_12K.zip'

--2021-04-16 11:20:42--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.73.240, 172.217.15.112, 172.217.164.144, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.73.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: ‘nature_12K.zip’

nature_12K.zip       60%[===========>        ]   2.14G  31.5MB/s    eta 17s    

In [ ]:
!unzip "/content/nature_12K.zip"
print("folders unzipped")

In [ ]:
cd "/content/inaturalist_12K/train"


In [ ]:
!rm -rf ".DS_Store"

In [ ]:
cd "/content"

##Dividing Data into train test and validation set

In [ ]:
import os
import shutil
training_folder_name = '/content/inaturalist_12K/train'

# New location for 
validation_folder = '/content/datasets/validation'

# Create the output folder if it doesn't already exist
if os.path.exists(validation_folder):
    shutil.rmtree(validation_folder)

# Loop through each subfolder in the input folder
print('Moving images...')
for root, folders, files in os.walk(training_folder_name):
    for sub_folder in folders:
        if sub_folder == ".DS_Store":
            continue
        print('processing folder ' + sub_folder)
        # Create a matching subfolder in the output dir
        saveFolder = os.path.join(validation_folder,sub_folder)
        if not os.path.exists(saveFolder):
            os.makedirs(saveFolder)
        # Loop through the files in the subfolder
        file_names = os.listdir(os.path.join(root,sub_folder))
        import random
        index=random.sample(range(0, len(file_names)), int(0.1*len(file_names)))
        for i in index:
            file_name = file_names[i]

            if file_name == ".DS_Store":
                continue
            file_path = os.path.join(root,sub_folder, file_name)
    
            shutil.move(file_path , saveFolder)

print('Done.')
print('renaming val folder as test folder')
#os.rename('/content/inaturalist_12K/val','/content/inaturalist_12K/test')
shutil.move('/content/inaturalist_12K/val' ,'/content/datasets/test')
shutil.move('/content/inaturalist_12K/train' ,'/content/datasets/train')

#**trainModel**

In [ ]:
def trainModel(model, train_loader, criterion, optimizer,scheduler, epoch, is_inception=False):
    scheduler.step()
    model.train()
    train_loss = 0
    print("Epoch:", epoch)
    running_loss = 0.0
    running_corrects = 0
    best_acc = 0.0
    # Process the images in batches
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        # Use the CPU or GPU as appropriate
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Reset the optimizer
        optimizer.zero_grad()
        with torch.set_grad_enabled(True):
            if is_inception:
                outputs, aux_outputs = model(inputs)
                loss1 = criterion(outputs, labels)
                loss2 = criterion(aux_outputs, labels)
                loss = loss1 + 0.3*loss2
            else:
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

            train_loss += loss.item()
            loss.backward()
            optimizer.step()
            #del inputs, labels, outputs
            torch.cuda.empty_cache()
        
        #statistics
        since = time.time()
       
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(train_loader.dataset)
    #wandb.log({"Train Loss": epoch_loss})
    epoch_acc = running_corrects.double() / len(train_loader.dataset)
    #wandb.log({"Train accuracy": epoch_acc})

    print('{} Loss: {:.4f} Acc: {:.4f}'.format('train', epoch_loss, epoch_acc))

    avg_loss = train_loss/(batch_idx+1)
    print('Training set: Average loss: {:.6f}'.format(avg_loss))
    #return model, avg_loss
    #return model,avg_loss
    return avg_loss,model
    

#Test Loader

In [ ]:
def testModel(model, device, test_loader,criterion):
    since = time.time()
    val_acc_history = []
    #best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    # Switch the model to evaluation mode ]
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    test_loss = 0
    correct = 0
    with torch.no_grad():
        batch_count = 0
        for inputs, labels in test_loader:
            batch_count += 1
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            # Calculate the loss for this batch
            loss = criterion(outputs, labels)
            test_loss += criterion(outputs, labels).item()
            
            # Calculate the accuracy for this batch
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(labels==preds).item()
            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

    # Calculate the average loss and total accuracy for this epoch
    epoch_loss = running_loss / len(test_loader.dataset)
    #wandb.log({"Val loss": epoch_loss})
    epoch_acc = running_corrects.double() / len(test_loader.dataset)
    avg_loss = test_loss / batch_count
    print('{} Loss: {:.4f} Acc: {:.4f}'.format('val', epoch_loss, epoch_acc))
    print('Validation set: Average loss: {:.6f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        avg_loss, running_corrects, len(test_loader.dataset),
        100. * running_corrects / len(test_loader.dataset)))

    #wandb.log({"Val accuracy": epoch_acc})


    return avg_loss,epoch_acc,model

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
  
    model_ft = None
    input_size = 0

    if model_name == "resnet18":
        """ Resnet18
        """
        print("resnet called")
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "resnet34":
        """ Resnet 34
        """
        model_ft = models.resnet34(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "resnet50":
        """ Resnet 50
        """
        model_ft = models.resnet50(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "resnet101":
        """ Resnet101
        """
        model_ft = models.resnet101(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "vgg11":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224
    elif model_name == "vgg19":
        """ VGG19_bn
        """
        model_ft = models.vgg19_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet121":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224
    elif model_name == "inceptionv3":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299
        
    elif model_name == "densenet161":
        """ densenet161
        """
        model_ft = models.densenet161(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224
    
    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size


#Dataset Transformation

In [ ]:
def Dataset_Transform(input_size=400,train_data_directory='/content/datasets/train',val_data_directory = '/content/datasets/validation',
                      test_data_directory = '/content/datasets/test',batch_size = 8):
        transform_train = transforms.Compose([
            transforms.RandomResizedCrop(input_size),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(degrees=15),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        transform_val = transforms.Compose([
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        transform_test = transforms.Compose([
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        train_set = torchvision.datasets.ImageFolder(
            root=train_data_directory,
            transform=transform_train
        )
        val_set = torchvision.datasets.ImageFolder(
            root=val_data_directory,
            transform=transform_val
        )
        test_set = torchvision.datasets.ImageFolder(
            root=test_data_directory,
            transform=transform_test
        )
        train_loader = torch.utils.data.DataLoader(
            train_set,
            batch_size=batch_size,
            num_workers=2,
            shuffle=True
        )
        val_loader = torch.utils.data.DataLoader(
            val_set,
            batch_size=batch_size,
            num_workers=2,
            shuffle=False
        )
        test_loader = torch.utils.data.DataLoader(
            test_set,
            batch_size=batch_size,
            num_workers=2,
            shuffle=False
        )
        dataloaders_dict = {}
        dataloaders_dict['train'] = train_loader
        dataloaders_dict['val'] = val_loader
        dataloaders_dict['test'] = test_loader

        return dataloaders_dict

## Function for selectively tuning layers

In [ ]:
 def unfreeze_layers(x,model_ft):
        layers =[]

        for name, child in model_ft.named_children():
            layers.append(name)
        index = range(-1,-1*min(x+1,len(layers)),-1)
        layers_to_unfreeze = []
        for id in index:
            layers_to_unfreeze.append(layers[id])
        for param in model_ft.parameters():
            param.requires_grad = True
        for name, child in model_ft.named_children():
            if name in  layers_to_unfreeze:
                print(name + ' is unfrozen')
                for param in child.parameters():
                    param.requires_grad = True
            else:
                print(name + ' is frozen')
                for param in child.parameters():
                    param.requires_grad = False
        return model_ft

#Fitting Model to device

In [ ]:
def fit_model_to_device(model_ft,device,feature_extract):
    # Send the model to GPU
    model_ft = model_ft.to(device)
    params_to_update = model_ft.parameters()
    print("Params to learn:")
    if feature_extract:
        params_to_update = []
        for name,param in model_ft.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t",name)
    else:
        for name,param in model_ft.named_parameters():
            if param.requires_grad == True:
                print("\t",name)
    return model_ft

#Fitting Model

In [ ]:
def fittinng_model(model_name,model_ft,optimizer,scheduler,criterion,epochs,dataloaders_dict):
    test_loader = dataloaders_dict['test']
    train_loader = dataloaders_dict['train']
    val_loader= dataloaders_dict['val']
    #print(criterion)
    # Track metrics in these arrays
    epoch_nums = []
    training_loss = []
    validation_loss = []
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model_ft.state_dict())
    print('Training on', device)
    for epoch in range(1, epochs + 1):
            #torch.cuda.empty_cache()
            train_loss,model_ft =  trainModel(model_ft, train_loader, criterion, optimizer, scheduler,epoch,is_inception=(model_name == "inceptionv3"))
            #train_loss,model_ft= trainModel(model_ft, device, train_loader, optimizer, epoch)
            val_loss,epoch_acc,model_ft = testModel(model_ft, device,val_loader,criterion)
            if(epoch_acc > best_acc):
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model_ft.state_dict())
            
            print('Best val Acc: {:4f}'.format(best_acc))
            epoch_nums.append(epoch)
            training_loss.append(train_loss)
            validation_loss.append(val_loss)
    # load best model weights
    model_ft.load_state_dict(best_model_wts)
    return  validation_loss, model_ft

#Function to evaluate train set

In [ ]:
def evaluateModel(dataloader, model,is_inception):
    total, correct = 0, 0
    for data in dataloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs= model(inputs)
        _, pred = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()
    return 100 * correct / total

#Initializing the model parameters

In [ ]:
#model_name ="inception"

In [ ]:
train_data_directory='/content/datasets/train'
val_data_directory = '/content/datasets/validation'
test_data_directory = '/content/datasets/test',
def startTraining(model_name, num_classess, batch_size, num_epochs,feature_extract,no_layers_to_unfreeze,lr,gamma,opt):

    model_ft, input_size = initialize_model(model_name=model_name, num_classes=10, feature_extract= feature_extract, use_pretrained=True)

    model_ft = model_ft.to(device)
    # Print the model we just instantiated
    #print(model_ft)

    dataloaders_dict=Dataset_Transform(input_size=input_size,train_data_directory='/content/datasets/train',val_data_directory = '/content/datasets/validation',
                        test_data_directory = '/content/datasets/test',batch_size = batch_size)
    model_ft = unfreeze_layers(no_layers_to_unfreeze,model_ft)
    model_ft = fit_model_to_device(model_ft,device,True)
    # Observe that all parameters are being optimized
    optimizer_ft  = None
    if(opt == "sgd"):
        # Decay LR by a factor of 0.1 every 7 epochs
        optimizer_ft = optim.SGD(model_ft.parameters(), lr=lr, momentum=0.9)
    elif(opt == "adam"):
        optimizer_ft = optim.Adam(model_ft.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.000001, amsgrad=False)
    elif(opt=="rmsp"):
        optimizer_ft = optim.RMSprop(model_ft.parameters(), lr=lr, alpha=0.99, eps=1e-08, weight_decay=0.0001, momentum=0.9, centered=False)
    elif(opt=="adagrad"):
        optimizer_ft = optim.Adagrad(model_ft.parameters(), lr=lr, lr_decay=0, weight_decay=0.0001, initial_accumulator_value=0, eps=1e-10)
    else:
        optimizer_ft = optim.SGD(model_ft.parameters(), lr=lr, momentum=0.9)

    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=gamma)

    # Specify the loss criteria
    criterion = nn.CrossEntropyLoss()
    validation_loss, model_ft = fittinng_model(model_name = model_name,model_ft=model_ft,optimizer=optimizer_ft,scheduler=exp_lr_scheduler,criterion=criterion,epochs=num_epochs,dataloaders_dict=dataloaders_dict)
    #Testing Model on Test Data Set
    Test_accuracy = evaluateModel(dataloaders_dict['test'], model_ft,is_inception=(model_name == "inceptionv3"))
    print('Test Accuracy: {:4f}'.format(Test_accuracy))
    return model_ft


#uncomment below part and comment wandb logs in testModel and trainModel function to run without wandb

In [ ]:
startTraining(model_name ="densenet161", num_classess =10, batch_size =32, num_epochs=2,feature_extract=False,no_layers_to_unfreeze=1,lr=0.001,gamma=0.1,opt = "sgd")

#Wandb Part

In [ ]:
sweep_config = {
    'name': 'Hyper_parameter_check',
    'method': 'random', #grid, random, bayes
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
    'parameters':{
        'model_name':{
           'values' :["inceptionv3","resnet50","alexnet", "vgg11", "squeezenet", "densenet161","resnet101"]
        },
        'max_epoch':{
            'values':[2,5,10]
        },
        'batch_size':{
           'values' :[32,64,128]
        },
        'layers_to_train':{
            'values':[1,2,3,4]
        },
        'learning_rate':{
            'values':[1e-2,1e-3,1e-4]
        },
        'gamma':{
            'values': [0.1,0.2,0.3]
        },
        'optimizer':{
            'values': ["adam","adagrad","sgd","rmsp"]
        }

    },
}


In [ ]:
wandb.login()
wandb.init(project = "DL_Assign_2B", entity = "cs20m038")
sweep_id = wandb.sweep(sweep_config, project='DL_Assign_2B', entity='cs20m038')

In [ ]:
def runSweep():
    run = wandb.init()
    configuration = run.config
    startTraining(model_name = configuration.model_name, num_classess = 10, batch_size = configuration.batch_size, num_epochs=configuration.max_epoch,feature_extract = False,no_layers_to_unfreeze=configuration.layers_to_train,lr=configuration.learning_rate,gamma=configuration.gamma,opt = configuration.optimizer)
    

    


In [ ]:
wandb.agent(sweep_id=sweep_id, function=runSweep)